# Joc del Nim

En aquesta pràctica implementarem el joc del Nim. El joc consisteix en un conjunt de piles de fitxes. En cada torn, un jugador ha de treure fitxes d'una de les piles. El jugador que es quedi sense fitxes en les piles perd. En aquesta pràctica implementarem una versió simplificada del joc, en la que només hi ha una pila.

La pila tindrà inicialment 20 fitxes i en cada torn el jugador pot treure 1, 2 o 3 fitxes. El jugador que es quedi sense fitxes perd. Per exemple, si en una partida hi ha dos jugadors, el primer jugador treu 3 fitxes i el segon jugador 2, el primer jugador perd perquè en la seva tirada es quedarà sense fitxes.

## Implementació del joc

### Representació de l'estat
En primer lloc necessitarem una forma de representar els diferents estats del joc. En cada estat definirem:

* `jugador`: el jugador que ha de fer el següent moviment.
* `utilitat`: mesura de la utilitat de l'estat pel jugador.
* `tauler`: estat del tauler del joc. En el nostre cas hi haurà prou amb saber el nombre de fitxes que queden a la pila
* `moviments`: els moviments possibles des de l'estat actual.
* `moviment`: el moviment que s'ha fet per arribar a l'estat actual.
* `estat_pare`: l'estat pare de l'estat actual.

In [365]:
from dataclasses import dataclass


@dataclass(frozen=True)
class EstatJoc:
    torn: str
    utilitat: int
    tauler: int
    moviments: list
    moviment: int = None
    estat_pare: 'EstatJoc' = None

### Classe `TresEnRatlla`

El nostres jocs heretaran de la classe `Joc` que implementa els mètodes bàsics per a la programació de jocs. Aquesta classe té els següents mètodes:

* `accions(self, estat)`: retorna els moviments possibles des de l'estat passat com a paràmetre.
* `resultat(self, estat, accio)`: retorna l'estat que resulta d'aplicar l'accio a l'estat.
* `es_terminal(self, estat)`: retorna `True` si l'estat és terminal.
* `utilitat(self, estat, jugador)`: retorna la utilitat de l'estat per al jugador.


In [366]:
class Joc:
    """Un joc és similar a un problema, però té una utilitat per a cada estat i un test terminal en lloc d'un cost de ruta i un test de l'objectiu. Per crear un joc, subclasseu aquesta classe i implementeu accions, resultats, utilitat, i test_terminal. Podeu sobreescriure la visualització i els successors o bé podeu heretar els seus mètodes per defecte. També necessitareu establir l'atribut .inicial a l'estat inicial; això es pot fer en el constructor."""

    def accions(self, estat):
        """Retorna una llista dels moviments permesos en aquest punt."""
        raise NotImplementedError

    def resultat(self, estat, moviment):
        """Retorna l'estat que resulta de fer un moviment des d'un estat."""
        raise NotImplementedError

    def utilitat(self, estat, jugador):
        """Retorna el valor d'aquest estat final per al jugador."""
        raise NotImplementedError

    def test_terminal(self, estat):
        """Retorna True si aquest és un estat final per al joc."""
        return not self.accions(estat)

    def torn(self, estat):
        """Retorna el jugador al qual li toca moure en aquest estat."""
        return estat.torn

    def display(self, estat):
        """Imprimeix o mostra d'alguna manera l'estat."""
        print(estat)

    def __repr__(self):
        return '<{}>'.format(self.__class__.__name__)

    def juga_joc(self, *jugadors):
        """Juga un joc de n-persones, alternant torns."""
        estat = self.inicial
        self.display(estat)
        while True:
            for jugador in jugadors:
                moviment = jugador(self, estat)
                estat = self.resultat(estat, moviment)
                self.display(estat)
                if self.test_terminal(estat):
                    self.display(estat)
                    print()
                    return self.utilitat(estat, self.torn(self.inicial))

Crearem una classe anomenada `JocNim` que herete de la classe `Joc`. Has de tindre en compte les següents consideracions:

* Els jugadors prenen torns alternats.
* El nombre de fitxes inicials és definida en el constructor de la classe; per defecte seran 20.
* Els moviments possibles també són definits en el constructor de la classe; per defecte seran 1, 2 i 3 fitxes.
* En cada torn s'ha de demanar al jugador que introdueixi el nombre de fitxes que vol treure. Si el moviment no és vàlid, s'ha de tornar a demanar al jugador que introdueixi un moviment vàlid.
* La utilitat serà 1 si guanya el jugador `'MAX'` (MIN ha segut l'últim en moure), -1 si guanya el jugador `'MIN' i 0 si la partida no ha acabat.
* L'estat inicial tindrà el jugador `'MAX'`, el tauler estarà buit i estaran disponibles totes les fitxes.

In [367]:
class JocNim(Joc):
    def __init__(self, n_fitxes=20, moviments=[1, 2, 3]):
        self.inicial = EstatJoc(
            torn='MAX',
            utilitat=0,
            tauler=n_fitxes,
            moviments=moviments,
            moviment=None,
            estat_pare=None
        )

    def accions(self, estat):
        return [
            moviment for moviment in estat.moviments
            if moviment <= estat.tauler
        ]

    def resultat(self, estat, moviment):
        return EstatJoc(
            torn='MAX' if estat.torn == 'MIN' else 'MIN',
            utilitat=self.utilitat(estat, estat.torn),
            tauler=estat.tauler - moviment,
            moviments=estat.moviments,
            moviment=moviment,
            estat_pare=estat
        )

    def utilitat(self, estat, jugador):
        if estat.tauler == 0:
            return 1 if jugador != estat.torn else -1
        return 0

    def display(self, estat):
        print('Torn:', estat.torn)
        print('Tauler:', estat.tauler)
        print('Moviments:', self.accions(estat))
        print('Moviment:', estat.moviment)
        print('Utilitat:', estat.utilitat)
        print("")

### `JugadorAleatori`

Per a poder jugar al joc necessitarem un jugador aleatori. Aquest jugador simplement escull un moviment aleatori entre els moviments possibles.

In [368]:
import random


def jugador_aleatori(joc, estat):
    return random.choice(joc.accions(estat))

Verifiquem el funcionament del joc amb dos jugadors aleatoris.

In [369]:
joc = JocNim()
joc.juga_joc(jugador_aleatori, jugador_aleatori)

Torn: MAX
Tauler: 20
Moviments: [1, 2, 3]
Moviment: None
Utilitat: 0

Torn: MIN
Tauler: 19
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0

Torn: MAX
Tauler: 18
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0

Torn: MIN
Tauler: 17
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0

Torn: MAX
Tauler: 15
Moviments: [1, 2, 3]
Moviment: 2
Utilitat: 0

Torn: MIN
Tauler: 12
Moviments: [1, 2, 3]
Moviment: 3
Utilitat: 0

Torn: MAX
Tauler: 10
Moviments: [1, 2, 3]
Moviment: 2
Utilitat: 0

Torn: MIN
Tauler: 8
Moviments: [1, 2, 3]
Moviment: 2
Utilitat: 0

Torn: MAX
Tauler: 7
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0

Torn: MIN
Tauler: 5
Moviments: [1, 2, 3]
Moviment: 2
Utilitat: 0

Torn: MAX
Tauler: 3
Moviments: [1, 2, 3]
Moviment: 2
Utilitat: 0

Torn: MIN
Tauler: 0
Moviments: []
Moviment: 3
Utilitat: 0

Torn: MIN
Tauler: 0
Moviments: []
Moviment: 3
Utilitat: 0



1

## Cerca adversària (minimax)

Per a que el joc sigui interessant necessitem un jugador intel·ligent. En aquest cas implementarem un jugador que utilitza l'algorisme minimax per a decidir el moviment a realitzar.

Per a implementar l'algorisme minimax necessitarem una funció d'avaluació que ens doni una mesura de la utilitat de l'estat per al jugador. En el nostre cas, la funció d'avaluació serà la mateixa utilitat de l'estat. Aquesta funció d'avaluació ens donarà un valor entre -1 i 1. Si el valor és 1, l'estat és guanyador per al jugador `'MAX'`, si el valor és -1, l'estat és guanyador per al jugador `'MIN'` i si el valor és 0, l'estat no és terminal.

Per a implementar l'algorisme minimax necessitarem dos funcions auxiliars:

* `maximitza(joc, estat)`: retorna la utilitat del moviment que té més utilitat per al jugador `'MAX'` i el moviment que l'ha generat.
* `minimitza(joc, estat)`: retorna la utilitat del moviment que té més utilitat per al jugador `'MIN'` i el moviment que l'ha generat.
* `minimax(joc, estat)`: retorna el moviment que té més utilitat per al jugador `'MAX'` (utilitzant les funcions `maximitza` i `minimitza`).


In [370]:
def minimax(joc, estat):
    def maximitza(joc, estat):
        if joc.test_terminal(estat):
            return joc.utilitat(estat, joc.torn(joc.inicial)), None
        v = -float('inf')
        moviment = None
        for accio in joc.accions(estat):
            v2, _ = minimitza(joc, joc.resultat(estat, accio))
            if v2 > v:
                v = v2
                moviment = accio
        return v, moviment

    def minimitza(joc, estat):
        if joc.test_terminal(estat):
            return joc.utilitat(estat, joc.torn(joc.inicial)), None
        v = float('inf')
        moviment = None
        for accio in joc.accions(estat):
            v2, _ = maximitza(joc, joc.resultat(estat, accio))
            if v2 < v:
                v = v2
                moviment = accio
        return v, moviment

    return maximitza(joc, estat)[1]

Testejem el funcionament de l'algorisme minimax amb el joc del Nim. 

In [371]:
joc = JocNim(n_fitxes=25)
joc.juga_joc(minimax, minimax)

Torn: MAX
Tauler: 25
Moviments: [1, 2, 3]
Moviment: None
Utilitat: 0
Torn: MIN
Tauler: 24
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0
Torn: MAX
Tauler: 21
Moviments: [1, 2, 3]
Moviment: 3
Utilitat: 0
Torn: MIN
Tauler: 20
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0
Torn: MAX
Tauler: 17
Moviments: [1, 2, 3]
Moviment: 3
Utilitat: 0

Torn: MIN
Tauler: 16
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0
Torn: MAX
Tauler: 13
Moviments: [1, 2, 3]
Moviment: 3
Utilitat: 0

Torn: MIN
Tauler: 12
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0

Torn: MAX
Tauler: 9
Moviments: [1, 2, 3]
Moviment: 3
Utilitat: 0

Torn: MIN
Tauler: 8
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0

Torn: MAX
Tauler: 5
Moviments: [1, 2, 3]
Moviment: 3
Utilitat: 0

Torn: MIN
Tauler: 4
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0

Torn: MAX
Tauler: 1
Moviments: [1]
Moviment: 3
Utilitat: 0

Torn: MIN
Tauler: 0
Moviments: []
Moviment: 1
Utilitat: 0

Torn: MIN
Tauler: 0
Moviments: []
Moviment: 1
Utilitat: 0



1

## Cerca adversària amb poda alfa-beta

L'algorisme minimax és molt eficient però no pràctic per a jocs amb moltes possibilitats. En aquest cas implementarem l'algorisme minimax amb poda alfa-beta. Aquest algorisme és una millora de l'algorisme minimax que permet reduir el nombre de nodes que s'han d'avaluar.

Per a implementar l'algorisme minimax amb poda alfa-beta necessitarem les següents funcions:

* `maximitza(joc, estat, alfa, beta)`: retorna el moviment que té més utilitat per al jugador `'MAX'`. `alfa` i `beta` són els valors de la millor opció trobada fins el moment per als jugadors `'MAX'` i `'MIN'` respectivament.
* `minimitza(joc, estat, alfa, beta)`: retorna el moviment que té menys utilitat per al jugador `'MAX'`. `alfa` i `beta` són els valors de la millor opció trobada fins el moment per als jugadors `'MAX'` i `'MIN'` respectivament.
* `minimax_ab(joc, estat)`: retorna el moviment que té més utilitat per al jugador `'MAX'`.


In [372]:
def minimax_ab(joc, estat):
    def maximitza(joc, estat, alfa, beta):
        if joc.test_terminal(estat):
            return joc.utilitat(estat, joc.torn(joc.inicial)), None
        v = -float('inf')
        moviment = None
        for accio in joc.accions(estat):
            v2, _ = minimitza(joc, joc.resultat(estat, accio), alfa, beta)
            if v2 > v:
                v = v2
                moviment = accio
            if v >= beta:
                return v, moviment
            alfa = max(alfa, v)
        return v, moviment

    def minimitza(joc, estat, alfa, beta):
        if joc.test_terminal(estat):
            return joc.utilitat(estat, joc.torn(joc.inicial)), None
        v = float('inf')
        moviment = None
        for accio in joc.accions(estat):
            v2, _ = maximitza(joc, joc.resultat(estat, accio), alfa, beta)
            if v2 < v:
                v = v2
                moviment = accio
            if v <= alfa:
                return v, moviment
            beta = min(beta, v)
        return v, moviment

    return maximitza(joc, estat, -float('inf'), float('inf'))[1]

Per a testar el funcionament de l'algorisme minimax amb poda alfa-beta jugarem una partida entre un jugador aleatori i un jugador que utilitza l'algorisme minimax amb poda alfa-beta en 25 fitxes.

In [373]:
joc = JocNim(n_fitxes=25)
joc.juga_joc(minimax_ab, minimax_ab)

Torn: MAX
Tauler: 25
Moviments: [1, 2, 3]
Moviment: None
Utilitat: 0
Torn: MIN
Tauler: 24
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0
Torn: MAX
Tauler: 21
Moviments: [1, 2, 3]
Moviment: 3
Utilitat: 0
Torn: MIN
Tauler: 20
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0

Torn: MAX
Tauler: 17
Moviments: [1, 2, 3]
Moviment: 3
Utilitat: 0
Torn: MIN
Tauler: 16
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0

Torn: MAX
Tauler: 13
Moviments: [1, 2, 3]
Moviment: 3
Utilitat: 0

Torn: MIN
Tauler: 12
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0

Torn: MAX
Tauler: 9
Moviments: [1, 2, 3]
Moviment: 3
Utilitat: 0

Torn: MIN
Tauler: 8
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0

Torn: MAX
Tauler: 5
Moviments: [1, 2, 3]
Moviment: 3
Utilitat: 0

Torn: MIN
Tauler: 4
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0

Torn: MAX
Tauler: 1
Moviments: [1]
Moviment: 3
Utilitat: 0

Torn: MIN
Tauler: 0
Moviments: []
Moviment: 1
Utilitat: 0

Torn: MIN
Tauler: 0
Moviments: []
Moviment: 1
Utilitat: 0



1

Com podem veure el rendiment de l'algorisme amb poda alfa-beta és molt millor que l'algorisme minimax. En aquest cas, l'algorisme amb poda alfa-beta té un rendiment més de 10 vegades millor que l'algorisme minimax:

* Algorisme minimax: 42.7 s ± 1.33 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
* Algorisme minimax amb poda alfa-beta: 3.41 s ± 124 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

## Funció minimax amb profunditat limitada i avaluació heurística

L'algorisme minimax amb poda alfa-beta és molt eficient però no pràctic per a jocs amb moltes possibilitats. En aquest cas implementarem l'algorisme minimax amb poda alfa-beta i profunditat limitada. Aquest algorisme és una millora de l'algorisme minimax amb poda alfa-beta que permet reduir el nombre de nodes que s'han d'avaluar. Quan s'arriba a la profunditat màxima s'avalua l'estat utilitzant una funció d'avaluació heurística.

Aquest algorisme no garanteix trobar el millor moviment però és molt més eficient i permet jugar a jocs amb moltes possibilitats.

La funció d'avaluació heurística determinarà la utilitat real de l'algorisme; es per això que és molt important que ens done uns resultats de qualitat. La funció haurà de preveure el guanyador de la partida i donar un valor alt a les situacions que siguin favorables per al jugador i un valor baix a les situacions que siguin desfavorables per al jugador.

Per a implementar l'algorisme minimax amb poda alfa-beta i profunditat limitada necessitarem les següents funcions:

* `maximitza(joc, estat, alfa, beta, profunditat)`: retorna el moviment que té més utilitat per al jugador `'MAX'`. `alfa` i `beta` són els valors de la millor opció trobada fins el moment per als jugadors `'MAX'` i `'MIN'` respectivament. `profunditat` és la profunditat màxima a explorar.
* `minimitza(joc, estat, alfa, beta, profunditat)`: retorna el moviment que té menys utilitat per al jugador `'MAX'`. `alfa` i `beta` són els valors de la millor opció trobada fins el moment per als jugadors `'MAX'` i `'MIN'` respectivament. `profunditat` és la profunditat màxima a explorar.
* `minimax_ab_prof(joc, estat, profunditat)`: retorna el moviment que té més utilitat per al jugador `'MAX'`.
* `avaluacio_heuristica(joc, estat)`: retorna la utilitat de l'estat per al jugador `'MAX'` utilitzant una funció heurística.
* minimax_ab_prof_h(joc, estat, profunditat)`: retorna el moviment que té més utilitat per al jugador `'MAX'` utilitzant una funció heurística si la profunditat d'avaluació és `> profunditat `.


In [374]:
def avaluacio_heuristica(joc, estat, profunditat):
    if joc.test_terminal(estat):
        return joc.utilitat(estat, joc.torn(joc.inicial))
    if profunditat == 0:
        return 0
    v = -float('inf')
    for accio in joc.accions(estat):
        v = max(v, avaluacio_heuristica(joc, joc.resultat(estat, accio), profunditat - 1))
    return v

def minimax_ab_prof_h(joc, estat, profunditat=3):
    def maximitza(joc, estat, alfa, beta, profunditat):
        if joc.test_terminal(estat):
            return joc.utilitat(estat, joc.torn(joc.inicial)), None
        if profunditat == 0:
            print(avaluacio_heuristica(joc, estat, profunditat))
            return avaluacio_heuristica(joc, estat, profunditat), None
        v = -float('inf')
        moviment = None
        for accio in joc.accions(estat):
            v2, _ = minimitza(joc, joc.resultat(estat, accio), alfa, beta, profunditat - 1)
            if v2 > v:
                v = v2
                moviment = accio
            if v >= beta:
                return v, moviment
            alfa = max(alfa, v)
        return v, moviment

    def minimitza(joc, estat, alfa, beta, profunditat):
        if joc.test_terminal(estat):
            return joc.utilitat(estat, joc.torn(joc.inicial)), None
        if profunditat == 0:
            return avaluacio_heuristica(joc, estat, profunditat), None
        v = float('inf')
        moviment = None
        for accio in joc.accions(estat):
            v2, _ = maximitza(joc, joc.resultat(estat, accio), alfa, beta, profunditat - 1)
            if v2 < v:
                v = v2
                moviment = accio
            if v <= alfa:
                return v, moviment
            beta = min(beta, v)
        return v, moviment

    return maximitza(joc, estat, -float('inf'), float('inf'), profunditat)[1]

Verifiquem el funcionament de l'algorisme minimax amb poda alfa-beta i profunditat limitada amb una profunditat de 3. Confrontarem minimax_ab_prof_h amb minimax_ab_prof en 10 partides i 25 fitxes; d'aquesta manera podrem veure si la funció heurística és efectiva.

In [375]:
results = []
for i in range(10):
    joc = JocNim(n_fitxes=25)
    result = joc.juga_joc(minimax_ab_prof_h, minimax_ab)
    results.append(result)

wins = [result == 1 for result in results]
print('Heuristic wins:', sum(wins))

Torn: MAX
Tauler: 25
Moviments: [1, 2, 3]
Moviment: None
Utilitat: 0

Torn: MIN
Tauler: 24
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0
Torn: MAX
Tauler: 21
Moviments: [1, 2, 3]
Moviment: 3
Utilitat: 0

Torn: MIN
Tauler: 20
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0

Torn: MAX
Tauler: 17
Moviments: [1, 2, 3]
Moviment: 3
Utilitat: 0

Torn: MIN
Tauler: 16
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0
Torn: MAX
Tauler: 13
Moviments: [1, 2, 3]
Moviment: 3
Utilitat: 0

Torn: MIN
Tauler: 12
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0

Torn: MAX
Tauler: 9
Moviments: [1, 2, 3]
Moviment: 3
Utilitat: 0

Torn: MIN
Tauler: 8
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0

Torn: MAX
Tauler: 5
Moviments: [1, 2, 3]
Moviment: 3
Utilitat: 0

Torn: MIN
Tauler: 4
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0

Torn: MAX
Tauler: 1
Moviments: [1]
Moviment: 3
Utilitat: 0

Torn: MIN
Tauler: 0
Moviments: []
Moviment: 1
Utilitat: 0

Torn: MIN
Tauler: 0
Moviments: []
Moviment: 1
Utilitat: 0


Torn: MAX
Tauler: 25

Hem guanyat les 10 partides amb la funció heurística. Això ens indica que la funció heurística és efectiva i ens permetrà afrontar jocs al molta més complexitat. Com a exemple podem jugar una partida amb 100 fitxes.

In [376]:
joc = JocNim(n_fitxes=100)
joc.juga_joc(minimax_ab_prof_h, minimax_ab_prof_h)

Torn: MAX
Tauler: 100
Moviments: [1, 2, 3]
Moviment: None
Utilitat: 0

Torn: MIN
Tauler: 99
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0

Torn: MAX
Tauler: 98
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0

Torn: MIN
Tauler: 97
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0

Torn: MAX
Tauler: 96
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0

Torn: MIN
Tauler: 95
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0

Torn: MAX
Tauler: 94
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0

Torn: MIN
Tauler: 93
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0

Torn: MAX
Tauler: 92
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0

Torn: MIN
Tauler: 91
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0

Torn: MAX
Tauler: 90
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0

Torn: MIN
Tauler: 89
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0

Torn: MAX
Tauler: 88
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0

Torn: MIN
Tauler: 87
Moviments: [1, 2, 3]
Moviment: 1
Utilitat: 0

Torn: MAX
Tauler: 86
Moviments: [1, 2, 3]
Moviment: 1
Util

1

## Jugador humà

Pots guanyar a la màquina? Implementa un jugador humà i juga una partida contra la màquina.

In [382]:
def jugador_huma(joc, estat):
    moviment = None
    while moviment not in joc.accions(estat):
        moviment = int(input(f'Hi ha {estat.tauler} fitxes. Quantes vols treure?{joc.accions(estat)} '))
    return moviment

joc = JocNim(n_fitxes=100, moviments=[1, 2, 3, 5, 10])
joc.juga_joc(jugador_huma, minimax_ab_prof_h)

Torn: MAX
Tauler: 100
Moviments: [1, 2, 3, 5, 10]
Moviment: None
Utilitat: 0
Torn: MIN
Tauler: 90
Moviments: [1, 2, 3, 5, 10]
Moviment: 10
Utilitat: 0

Torn: MAX
Tauler: 89
Moviments: [1, 2, 3, 5, 10]
Moviment: 1
Utilitat: 0

Torn: MIN
Tauler: 79
Moviments: [1, 2, 3, 5, 10]
Moviment: 10
Utilitat: 0

Torn: MAX
Tauler: 78
Moviments: [1, 2, 3, 5, 10]
Moviment: 1
Utilitat: 0
Torn: MIN
Tauler: 68
Moviments: [1, 2, 3, 5, 10]
Moviment: 10
Utilitat: 0

Torn: MAX
Tauler: 67
Moviments: [1, 2, 3, 5, 10]
Moviment: 1
Utilitat: 0

Torn: MIN
Tauler: 57
Moviments: [1, 2, 3, 5, 10]
Moviment: 10
Utilitat: 0

Torn: MAX
Tauler: 56
Moviments: [1, 2, 3, 5, 10]
Moviment: 1
Utilitat: 0
Torn: MIN
Tauler: 46
Moviments: [1, 2, 3, 5, 10]
Moviment: 10
Utilitat: 0

Torn: MAX
Tauler: 45
Moviments: [1, 2, 3, 5, 10]
Moviment: 1
Utilitat: 0
Torn: MIN
Tauler: 35
Moviments: [1, 2, 3, 5, 10]
Moviment: 10
Utilitat: 0

Torn: MAX
Tauler: 34
Moviments: [1, 2, 3, 5, 10]
Moviment: 1
Utilitat: 0
Torn: MIN
Tauler: 24
Moviments: [

1